### Imports

In [82]:
from bs4 import BeautifulSoup
import requests
import json
import re
import string
import nltk
from pymongo import MongoClient
from nltk.corpus import stopwords
import pyarabic.araby as araby
from pyarabic.number import ArNumbers
from tashaphyne.stemming import ArabicLightStemmer
import qalsadi.lemmatizer as lem
from farasa.pos import FarasaPOSTagger 
from farasa.ner import FarasaNamedEntityRecognizer

### Loading the BeautifulSoup object

In [83]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36 Edg/123.0.0.0'}
result = requests.get('https://www.hespress.com/economie', headers=headers)
doc = BeautifulSoup(result.text, 'html.parser')

### Retrieving the divs that contain the links to articles

In [84]:
main_div = doc.find_all(name='div',attrs={'class':'posts-categoy'})
posts = main_div[0].find_all(name='div',attrs={'class':'card-img-top'})
articals = []
for article in posts:
    # links
    link = article.find(name='a').get('href')
    # article titles
    title = article.find(name='a').get('title')
    articals.append({title : link})

### Storing links and titles in a json file

In [85]:
with open('hess_article_links.json', 'w', encoding='utf-8') as f:
    data = []
    for article in articals:
        title = list(article.keys())[0]
        link = list(article.values())[0]
        data.append({'title': title, 'link': link})
    json.dump(data, f, ensure_ascii=False, indent=2)

### Retrieving the divs that contain the links to articles

In [86]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f:
    article_content = []
    data = json.load(f)
    for article in data:
        article_result = requests.get(article['link'], headers=headers)
        article_doc = BeautifulSoup(article_result.text, 'html.parser')
        article_content.append(article_doc.find_all(name='div',attrs={'class':'article-content'})[0].find_all(name='p'))

### Storing articles and their respective titles in a json file

In [87]:
with open('hess_article_content.json', 'w', encoding='utf-8') as f:
    contents = []
    for i,content in enumerate(article_content):
        content = ''
        for p in article_content[i]:
            content += p.text
        contents.append({'title': list(data[i].values())[0], 'content': content})
    json.dump(contents, f, ensure_ascii=False, indent=2)

### Uploading the two json files in a MongoDB database

In [88]:
with open('hess_article_links.json', 'r', encoding='utf-8') as f1:
    links = json.load(f1)
with open('hess_article_content.json', 'r', encoding='utf-8') as f2:
    art_content = json.load(f2)
client = MongoClient('mongodb://localhost:27017/')
db = client['NLP_lab1']
coll1 = db['hess_article_links']
coll2 = db['hess_article_content']
coll1.drop()
coll1.insert_many(links)
coll2.drop()
coll2.insert_many(art_content)
client.close()

### Loading the first article for the NLP pipeline

In [89]:
text = art_content[0]['content']
text.encode('utf-8')
print("Raw text: ",text)

Raw text:  أعلنت شركة الطيران الإيرلندية “ريان إير” أنها نقلت 13.6 مليون مسافر في شهر مارس الماضي، بزيادة قدرها 8 في المائة مقارنة بشهر مارس 2023.وقامت الشركة بتشغيل 77 ألف رحلة خلال شهر مارس بنسبة ملء وصلت 93 في المائة، موضحة أن 950 رحلة ألغيت بسبب النزاعات المسلحة.وأشارت الشركة إلى أنها نقلت، بين مارس 2023 ومارس 2024، ما مجموعه 183.7 مليون مسافر (+9 في المائة مقارنة بالفترة نفسها من العام السابق) بمعدل ملء قدره 94 في المائة (+1 نقطة).


### Tokenization

In [90]:
tokens = araby.tokenize(text)
sent_tokens = araby.sentence_tokenize(text)
print("Word tokenized list: ", tokens)
print("Sentence tokenized list: ", sent_tokens)
print("Word tokenized list length: ", len(tokens))
print("Sentence tokenized list length: ", len(sent_tokens))

Word tokenized list:  ['أعلنت', 'شركة', 'الطيران', 'الإيرلندية', '“', 'ريان', 'إير', '”', 'أنها', 'نقلت', '13', '.', '6', 'مليون', 'مسافر', 'في', 'شهر', 'مارس', 'الماضي', '،', 'بزيادة', 'قدرها', '8', 'في', 'المائة', 'مقارنة', 'بشهر', 'مارس', '2023', '.', 'وقامت', 'الشركة', 'بتشغيل', '77', 'ألف', 'رحلة', 'خلال', 'شهر', 'مارس', 'بنسبة', 'ملء', 'وصلت', '93', 'في', 'المائة', '،', 'موضحة', 'أن', '950', 'رحلة', 'ألغيت', 'بسبب', 'النزاعات', 'المسلحة', '.', 'وأشارت', 'الشركة', 'إلى', 'أنها', 'نقلت', '،', 'بين', 'مارس', '2023', 'ومارس', '2024', '،', 'ما', 'مجموعه', '183', '.', '7', 'مليون', 'مسافر', '(+', '9', 'في', 'المائة', 'مقارنة', 'بالفترة', 'نفسها', 'من', 'العام', 'السابق', ')', 'بمعدل', 'ملء', 'قدره', '94', 'في', 'المائة', '(+', '1', 'نقطة', ').']
Sentence tokenized list:  ['أعلنت شركة الطيران الإيرلندية “ريان إير” أنها نقلت 13.6 مليون مسافر في شهر مارس الماضي،', 'بزيادة قدرها 8 في المائة مقارنة بشهر مارس 2023.وقامت الشركة بتشغيل 77 ألف رحلة خلال شهر مارس بنسبة ملء وصلت 93 في المائة،', '

### Removing punctuation

In [91]:
ar_punct = ''')(+`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”،.”…“–ـ”.'''
en_punct = string.punctuation
punct_lst = ar_punct + en_punct

In [92]:
tokens = [token for token in tokens if token not in punct_lst]
sent_tokens = [sent.translate(str.maketrans('', '', punct_lst)) for sent in sent_tokens]
print("Tokenized text without punctuation: ",tokens)
print("Sentence tokenized text without punctuation: ",sent_tokens)
print("Length of tokenized text without punctuation: ", len(tokens))
print("Length of sentence tokenized text without punctuation: ", len(sent_tokens))

Tokenized text without punctuation:  ['أعلنت', 'شركة', 'الطيران', 'الإيرلندية', 'ريان', 'إير', 'أنها', 'نقلت', '13', '6', 'مليون', 'مسافر', 'في', 'شهر', 'مارس', 'الماضي', 'بزيادة', 'قدرها', '8', 'في', 'المائة', 'مقارنة', 'بشهر', 'مارس', '2023', 'وقامت', 'الشركة', 'بتشغيل', '77', 'ألف', 'رحلة', 'خلال', 'شهر', 'مارس', 'بنسبة', 'ملء', 'وصلت', '93', 'في', 'المائة', 'موضحة', 'أن', '950', 'رحلة', 'ألغيت', 'بسبب', 'النزاعات', 'المسلحة', 'وأشارت', 'الشركة', 'إلى', 'أنها', 'نقلت', 'بين', 'مارس', '2023', 'ومارس', '2024', 'ما', 'مجموعه', '183', '7', 'مليون', 'مسافر', '9', 'في', 'المائة', 'مقارنة', 'بالفترة', 'نفسها', 'من', 'العام', 'السابق', 'بمعدل', 'ملء', 'قدره', '94', 'في', 'المائة', '1', 'نقطة', ').']
Sentence tokenized text without punctuation:  ['أعلنت شركة الطيران الإيرلندية ريان إير أنها نقلت 136 مليون مسافر في شهر مارس الماضي', 'بزيادة قدرها 8 في المائة مقارنة بشهر مارس 2023وقامت الشركة بتشغيل 77 ألف رحلة خلال شهر مارس بنسبة ملء وصلت 93 في المائة', 'موضحة أن 950 رحلة ألغيت بسبب النزاعات 

### Removing stopwords

In [93]:
stop_words = set(stopwords.words('arabic'))
print("Stop words: ",stop_words)
tokens = [word for word in tokens if word not in stop_words]
sent_tokens = [''.join([x for x in re.split(r'(\W+)', sent) if x not in stop_words]) for sent in sent_tokens]

print("Tokenized text without stopwords: ",tokens)
print("Sentence tokenized text without stopwords: ",sent_tokens)
print("Length of tokenized text without stopwords: ", len(tokens))
print("Length of sentence tokenized text without stopwords: ", len(sent_tokens))

Stop words:  {'أنشأ', 'بماذا', 'كيت', 'فيما', 'إمّا', 'عاشر', 'أقبل', 'جنيه', 'سين', 'هَذِي', 'بهن', 'بين', 'ذِي', 'هَاتِه', 'لهم', 'بس', 'ثمانية', 'تسعمئة', 'هلم', 'شباط', 'آ', 'إذ', 'غالبا', 'أفٍّ', 'مكانكم', 'حيثما', 'قاطبة', 'اثنا', 'أيّان', 'كما', 'طالما', 'هيهات', 'ف', 'تاسع', 'خمس', 'أمام', 'حيث', 'بَلْهَ', 'حبذا', 'مكانَك', 'الألى', 'تانِ', 'ة', 'ثالث', 'اللاتي', 'فإذا', 'ح', 'شتانَ', 'لكيلا', 'إذاً', 'ثمان', 'كأن', 'إياها', 'كأنما', 'أيضا', 'بغتة', 'أوت', 'ذَيْنِ', 'دواليك', 'أمسى', 'سوى', 'س', 'صبر', 'هَذانِ', 'دونك', 'مه', 'اللتيا', 'لنا', 'لكم', 'قبل', 'قلما', 'بك', 'إنَّ', 'معاذ', 'سبعة', 'منها', 'نَّ', 'ثمّ', 'ليت', 'اربعين', 'هذا', 'نوفمبر', 'ته', 'أما', 'آنفا', 'هَذَيْنِ', 'إلا', 'إحدى', 'تعسا', 'مائة', 'حقا', 'جيم', 'تانِك', 'إي', 'هكذا', 'لن', 'زعم', 'هَيْهات', 'يفعلان', 'أجمع', 'ممن', 'خ', 'أربعمئة', 'هذين', 'هما', 'حدَث', 'لستن', 'لهما', 'م', 'ى', 'حتى', 'حَذارِ', 'جويلية', 'صهْ', 'آناء', 'فاء', 'فوق', 'كلّما', 'درى', 'رجع', 'بهم', 'هنا', 'إياهما', 'هلّا', 'اللتان',

### Convert numbers to words

In [94]:
arnum = ArNumbers()
num_tokens = []
for i,token in enumerate(tokens):
    if tokens[i].isdigit():
        tokens[i] = arnum.int2str(tokens[i])
        num_tokens.append(tokens[i])
print("Number to string: ", tokens)
print("Numerical tokens: ", num_tokens)

Number to string:  ['أعلنت', 'شركة', 'الطيران', 'الإيرلندية', 'ريان', 'إير', 'أنها', 'نقلت', 'ثلاث عشرة', 'ست', 'مليون', 'مسافر', 'شهر', 'الماضي', 'بزيادة', 'قدرها', 'ثماني', 'المائة', 'مقارنة', 'بشهر', 'ألفان و ثلاث و عشرون', 'وقامت', 'الشركة', 'بتشغيل', 'سبع و سبعون', 'رحلة', 'خلال', 'شهر', 'بنسبة', 'ملء', 'وصلت', 'ثلاث و تسعون', 'المائة', 'موضحة', 'تسعمئة و خمسون', 'رحلة', 'ألغيت', 'بسبب', 'النزاعات', 'المسلحة', 'وأشارت', 'الشركة', 'أنها', 'نقلت', 'ألفان و ثلاث و عشرون', 'ومارس', 'ألفان و أربع و عشرون', 'مجموعه', 'مئة و ثلاث و ثمانون', 'سبع', 'مليون', 'مسافر', 'تسع', 'المائة', 'مقارنة', 'بالفترة', 'نفسها', 'العام', 'السابق', 'بمعدل', 'ملء', 'قدره', 'أربع و تسعون', 'المائة', 'واحد', 'نقطة', ').']
Numerical tokens:  ['ثلاث عشرة', 'ست', 'ثماني', 'ألفان و ثلاث و عشرون', 'سبع و سبعون', 'ثلاث و تسعون', 'تسعمئة و خمسون', 'ألفان و ثلاث و عشرون', 'ألفان و أربع و عشرون', 'مئة و ثلاث و ثمانون', 'سبع', 'تسع', 'أربع و تسعون', 'واحد']


### Normalization

In [95]:
ArListem = ArabicLightStemmer()
# removing numerical tokens
for i in num_tokens:
    tokens.remove(i)
tokens = [ArListem.normalize(token) for token in tokens]
print("Normalized token list: ", tokens)

Normalized token list:  ['اعلنت', 'شركه', 'الطيران', 'الايرلنديه', 'ريان', 'اير', 'انها', 'نقلت', 'مليون', 'مسافر', 'شهر', 'الماضي', 'بزياده', 'قدرها', 'الماءه', 'مقارنه', 'بشهر', 'وقامت', 'الشركه', 'بتشغيل', 'رحله', 'خلال', 'شهر', 'بنسبه', 'ملء', 'وصلت', 'الماءه', 'موضحه', 'رحله', 'الغيت', 'بسبب', 'النزاعات', 'المسلحه', 'واشارت', 'الشركه', 'انها', 'نقلت', 'ومارس', 'مجموعه', 'مليون', 'مسافر', 'الماءه', 'مقارنه', 'بالفتره', 'نفسها', 'العام', 'السابق', 'بمعدل', 'ملء', 'قدره', 'الماءه', 'نقطه', ').']


### Stemming

In [96]:
stemmed_tokens = [ArListem.light_stem(token) for token in tokens]
print("Stemmed token list: ", stemmed_tokens)

Stemmed token list:  ['اعلن', 'شرك', 'طير', 'الايرلنديه', 'ري', 'ير', 'نه', 'قلت', 'ملي', 'مسافر', 'شهر', 'ماض', 'زياد', 'قدر', 'ماءه', 'مقار', 'شهر', 'قام', 'شركه', 'تشغيل', 'رحل', 'خلال', 'شهر', 'نسب', 'ملء', 'صل', 'ماءه', 'موضح', 'رحل', 'غيت', 'سبب', 'نزاع', 'مسلحه', 'اشار', 'شركه', 'نه', 'قلت', 'مارس', 'مجموع', 'ملي', 'مسافر', 'ماءه', 'مقار', 'فتره', 'نفس', 'عام', 'سابق', 'معدل', 'ملء', 'قدر', 'ماءه', 'قط', ').']


### Lemmatization

In [97]:
lemmer = lem.Lemmatizer()
lemmatized_tokens = [lemmer.lemmatize(token) for token in tokens]
print("Lemmatized token list: ", lemmatized_tokens)

Lemmatized token list:  ['اعلنت', 'شرك', 'طير', 'الايرلنديه', 'ري', 'وري', 'انها', 'نقل', 'مليون', 'مسافر', 'شهر', 'ماضي', 'زياد', 'قدر', 'الماءه', 'مقارن', 'شهر', 'قام', 'الشركه', 'تشغيل', 'رحل', 'خلال', 'شهر', 'نسب', 'ملء', 'صلت', 'الماءه', 'موضح', 'رحل', 'الغيت', 'سبب', 'نزاع', 'المسلحه', 'واشارت', 'الشركه', 'انها', 'نقل', 'مارس', 'مجموع', 'مليون', 'مسافر', 'الماءه', 'مقارن', 'بالفتره', 'نفس', 'عام', 'سابق', 'معدل', 'ملء', 'قدر', 'الماءه', 'نقط', ').']


### Stemming vs lemmatization comparison

In [98]:
print("Original token list: ", tokens)
print("Stemmed token list: ", stemmed_tokens)
print("Lemmatized token list: ", lemmatized_tokens)

Original token list:  ['اعلنت', 'شركه', 'الطيران', 'الايرلنديه', 'ريان', 'اير', 'انها', 'نقلت', 'مليون', 'مسافر', 'شهر', 'الماضي', 'بزياده', 'قدرها', 'الماءه', 'مقارنه', 'بشهر', 'وقامت', 'الشركه', 'بتشغيل', 'رحله', 'خلال', 'شهر', 'بنسبه', 'ملء', 'وصلت', 'الماءه', 'موضحه', 'رحله', 'الغيت', 'بسبب', 'النزاعات', 'المسلحه', 'واشارت', 'الشركه', 'انها', 'نقلت', 'ومارس', 'مجموعه', 'مليون', 'مسافر', 'الماءه', 'مقارنه', 'بالفتره', 'نفسها', 'العام', 'السابق', 'بمعدل', 'ملء', 'قدره', 'الماءه', 'نقطه', ').']
Stemmed token list:  ['اعلن', 'شرك', 'طير', 'الايرلنديه', 'ري', 'ير', 'نه', 'قلت', 'ملي', 'مسافر', 'شهر', 'ماض', 'زياد', 'قدر', 'ماءه', 'مقار', 'شهر', 'قام', 'شركه', 'تشغيل', 'رحل', 'خلال', 'شهر', 'نسب', 'ملء', 'صل', 'ماءه', 'موضح', 'رحل', 'غيت', 'سبب', 'نزاع', 'مسلحه', 'اشار', 'شركه', 'نه', 'قلت', 'مارس', 'مجموع', 'ملي', 'مسافر', 'ماءه', 'مقار', 'فتره', 'نفس', 'عام', 'سابق', 'معدل', 'ملء', 'قدر', 'ماءه', 'قط', ').']
Lemmatized token list:  ['اعلنت', 'شرك', 'طير', 'الايرلنديه', 'ري', 'وري', 'ان

### Machine learning approach for PoS tagging

In [106]:
pos_tagger = FarasaPOSTagger()

In [107]:
tagged_text =  pos_tagger.tag(tokens[3]) # [pos_tagger.tag(token) for token in lemmatized_tokens]
print("POS Tagged",tagged_text)

POS Tagged S/S ال+ ايرلنديه/DET+NOUN-MS E/E


In [ ]:
pos_tagger.terminate()

### Rule-Based approach for PoS tagging

In [156]:
def tag(token):
    rules_1_2 = [r'^\u0643\u0627\u0644.*$', r'^\u0628\u0627\u0644.*$', r'^\u0641\u0627\u0644.*$', 
                r'^\u0648\u0627\u0644.*$',r'\w*\u0627\u0626\u0647\b', r'\w*\u0627\u0626\u0643\b',
                r'\w*\u0627\u0626\u064a\b', r'\w*\u0627\u0624\u0643\b', r'\w*\u0627\u0624\u0647\b',
                r'\w*\u0627\u0621\u0643\b',r'\w*\u0627\u0621\u0647\b',r'\w*\u0647\u0645\u0627\b',
                r'\w*\u0643\u0645\u0627\b']
    rules_3_4 = [r'^\u0633\u064a.*$', r'^\u0633\u062a.*$', r'^\u0633\u0646.*$', r'^\u0633\u0623.*$',
                r'^\u0633\u0627.*$',r'^\u0644\u0627.*$',r'^\u0644\u0623.*$',r'^\u0644\u0646.*$',
                r'^\u0644\u062a.*$',r'^\u0644\u064a.*$', r'\w*\u064a\b',r'\w*\u0647\b',r'\w*\u0643\b',
                r'\w*\u0627\b',r'\w*\u0646\b',r'\w*\u0648\b']
    rules_5_6 = [r'^\w{3}.*\u0649$', r'^\w{2}\u0648\w{1}$',r'^\w{2}\u0627\u0621$',r'^\w*\u0627\u062a$']
    rule_7 = r'^(\u064a|\u0646)\w*(\u0648\u0646|\u064a\u0646)$'
    rules_8_9 = [r'^[^(\u064a|\u0646)]\w*(\u0648\u0646|\u064a\u0646)$']
    if any(re.match(pattern, token) for pattern in rules_1_2):
        return 'NOUN'
    elif any(re.match(pattern, token) for pattern in rules_3_4):
        return 'VERB'
    elif any(re.match(pattern, token) for pattern in rules_5_6):
        return 'NOUN'
    elif re.match(rule_7, token):
        return 'VERB'
    elif any(re.match(pattern, token) for pattern in rules_8_9):
        return 'NOUN'
    
print(tag('فعول'))

NOUN


## References
- Zerrouki, T., (2023). PyArabic: A Python package for Arabic text. Journal of Open Source Software, 8(84), 4886, https://doi.org/10.21105/joss.04886
- Alkhatib, R. M., Zerrouki, T., Shquier, M. M. A., & Balla, A. (2023). Tashaphyne0.4: A new arabic light stemmer based on rhyzome modeling approach. Information Retrieval Journa, 26(14). doi: https://doi.org/10.1007/s10791-023-09429-y
- T. Zerrouki, Qalsadi, Arabic mophological analyzer Library for python.,  https://pypi.python.org/pypi/qalsadi/
- Hegazi, M. O., Al-Dossari, Y., Al-Yahy, A., Al-Sumari, A., & Hilal, A. (2021). Preprocessing Arabic text on social media. Heliyon, 7(2), e06191. doi:10.1016/j.heliyon.2021.e06191 
- Sawalha, M., Atwell, E., & Abushariah, M. A. M. (2013). SALMA: Standard Arabic Language Morphological Analysis. 2013 1st International Conference on Communications, Signal Processing, and Their Applications (ICCSPA). doi:10.1109/iccspa.2013.6487311 